# smallest-free-number
A coding challenge to find the smallest non-negative integer that is not part of a given set of non-negative integers.

Thanks to [Nicolas Rinaudo](https://github.com/abcoates/smallest-free-number.git) for suggesting this problem for a coding challenge.

## Description
This is a simplification a general problem, find the 'least XXX' object that is not already used, where 'XXX' is some arbitrary measurement dimension such as 'large', 'expensive', etc.

In this minimal version, you are given a set (i.e. an **unordered** set) of non-negative integers, and you have to find the smallest non-negative integer that is **not** a part of the set.

## Examples
 * \[0, 1, 2, 3, 5\] => 4
 * \[0, 1, 3, 4, 5\] => 2
 * \[2, 1, 0\] => 3
 * \[20, 10, 30\] => 0

## Special Note
You **may** use AI to help you write the code.  As AI coding companions are inevitable, we might as well all start practicising how to use them.  **However**, if you have used AI to help you write your code, please make that clear in your solution.

## Stretch Goal
Finding a solution is straightforward, but can you find a solution that only takes linear time?  Measure how the time taken for your solution varies as the size of the set is increased, and see how close you can get to the time taken growing linearly with the size of the set.  Create a graph of the time measurements again set size.

You will need to generate unordered sets of numbers that are sufficiently large to make the solution time sufficiently measurable.

Your results may vary with the **density** of the sets, i.e. with the percentage of unused numbers in the range from zero to the largest number in the set.

## Solution - F#

### Solution #1 - Brute Force

As a first solution, let's try something "brute force" - sort the set, then walk up the integers from zero until we find one that isn't in the set.

In [142]:
let solution1 (intset: int list): int = // 'intset' must be a set in unordered list format - to avoid pre-sorted F# sets giving an advantage
    let sortedset = intset |> List.sort
    let rec find (sortedlist: int list) (nextValue: int) =
        match sortedlist with
        | [] -> nextValue
        | other ->
            let listhead = sortedlist |> List.head
            if (nextValue < listhead)
            then nextValue
            elif (nextValue = listhead)
            then find (sortedlist |> List.tail) (nextValue+1)
            else find(sortedlist |> List.tail) nextValue
    find sortedset 0

\[0, 1, 2, 3, 5\] => 4

In [143]:
solution1 [0;1;2;3;5]

4

\[0, 1, 3, 4, 5\] => 2

In [144]:
solution1 [0;1;3;4;5]

2

\[2, 1, 0\] => 3

In [145]:
solution1 [0;1;2]

3

\[20, 10, 30\] => 0

In [146]:
solution1 [10;20;30]

0

So 'solution1' works correctly.  However, how performant is it?

We'll need a graphing package - 'XPlot' will do nicely.

In [147]:
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"

open XPlot.Plotly

Installed Packages XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

Let's also get a package for curve fitting.

In [148]:
#r "nuget: MathNet.Numerics"

open MathNet.Numerics

Installed Packages MathNet.Numerics, 5.0.0

We'll need functions that can generate random sets of non-negative integers.

**However**, F# seems to implement sets as sorted lists, which makes it too easy to find the smallest unused integer.  As such, I will need to return the set as a list in which the order of the elements has been randomised.

In [149]:
open System

let randomGenerator = Random()

In [150]:
let rec randomiseList (aList: 'a list): 'a list =
    match aList with
    | [] -> aList
    | head::[] -> aList
    | _ ->
        let skipSize = randomGenerator.Next(0, aList.Length)
        let start = aList |> List.take skipSize
        let remainder = aList |> List.skip skipSize
        let chosen = remainder |> List.head
        let rest = remainder |> List.tail
        chosen::(randomiseList(List.concat [start; rest]))

In [151]:
randomiseList [1;2;3;4;5;6;7;8;9;10]

[ 6, 1, 9, 5, 7, 8, 3, 2, 10, 4 ] HeadOrDefault 6 TailOrNull [ 1, 9, 5, 7, 8, 3, 2, 10, 4 ] HeadOrDefault 1 TailOrNull [ 9, 5, 7, 8, 3, 2, 10, 4 ] HeadOrDefault 9 TailOrNull [ 5, 7, 8, 3, 2, 10, 4 ] HeadOrDefault 5 TailOrNull [ 7, 8, 3, 2, 10, 4 ] HeadOrDefault 7 TailOrNull [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] Head 7 Tail [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] (values) [ 7, 8, 3, 2, 10, 4 ] Head 5 Tail [ 7, 8, 3, 2, 10, 4 ] HeadOrDefault 7 TailOrNull [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] Head 7 Tail [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] (values) [ 7, 8, 3, 2, 10, 4 ] (values) [ 5, 7, 8, 3, 2, 10, 4 ] Head 9 Tail [ 5, 7, 8, 3, 2, 10, 4 ] HeadOrDefault 5 TailOrNull [ 7, 8, 3, 2, 10, 4 ] HeadOrDefault 7 TailOrNull [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] Head 7 Tail [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] (values) [ 7, 8, 3, 2, 10, 4 ] Head 5 Tail [ 7, 8, 3, 2, 10, 4 ] HeadOrDefault 7 TailOrNull [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] Head 7 Tail [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] (values) [ 7, 8, 3, 2, 10, 4 ] (values) [ 5, 7, 8, 3, 2, 10, 4 ] (values) [ 9, 5, 7, 8, 3, 2, 10, 4 ] Head 1 Tail [ 9, 5, 7, 8, 3, 2, 10, 4 ] HeadOrDefault 9 TailOrNull [ 5, 7, 8, 3, 2, 10, 4 ] HeadOrDefault 5 TailOrNull [ 7, 8, 3, 2, 10, 4 ] HeadOrDefault 7 TailOrNull [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] Head 7 Tail [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] (values) [ 7, 8, 3, 2, 10, 4 ] Head 5 Tail [ 7, 8, 3, 2, 10, 4 ] HeadOrDefault 7 TailOrNull [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] Head 7 Tail [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] (values) [ 7, 8, 3, 2, 10, 4 ] (values) [ 5, 7, 8, 3, 2, 10, 4 ] Head 9 Tail [ 5, 7, 8, 3, 2, 10, 4 ] HeadOrDefault 5 TailOrNull [ 7, 8, 3, 2, 10, 4 ] HeadOrDefault 7 TailOrNull [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] Head 7 Tail [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] (values) [ 7, 8, 3, 2, 10, 4 ] Head 5 Tail [ 7, 8, 3, 2, 10, 4 ] HeadOrDefault 7 TailOrNull [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] Head 7 Tail [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] (values) [ 7, 8, 3, 2, 10, 4 ] (values) [ 5, 7, 8, 3, 2, 10, 4 ] (values) [ 9, 5, 7, 8, 3, 2, 10, 4 ] (values) [ 1, 9, 5, 7, 8, 3, 2, 10, 4 ] Head 6 Tail [ 1, 9, 5, 7, 8, 3, 2, 10, 4 ] HeadOrDefault 1 TailOrNull [ 9, 5, 7, 8, 3, 2, 10, 4 ] HeadOrDefault 9 TailOrNull [ 5, 7, 8, 3, 2, 10, 4 ] HeadOrDefault 5 TailOrNull [ 7, 8, 3, 2, 10, 4 ] HeadOrDefault 7 TailOrNull [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] Head 7 Tail [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 TailOrNull [ 3, 2, 10, 4 ] Head 8 Tail [ 3, 2, 10, 4 ] (values) [ 8, 3, 2, 10, 4 ] (values) [ 7, 8, 3, 2, 10, 4 ] Head 5 Tail [ 7, 8, 3, 2, 10, 4 ] HeadOrDefault 7 TailOrNull [ 8, 3, 2, 10, 4 ] HeadOrDefault 8 Tail

In [152]:
randomiseList [1;2;3;4;5;6;7;8;9;10]

[ 5, 7, 10, 1, 2, 6, 3, 4, 9, 8 ] HeadOrDefault 5 TailOrNull [ 7, 10, 1, 2, 6, 3, 4, 9, 8 ] HeadOrDefault 7 TailOrNull [ 10, 1, 2, 6, 3, 4, 9, 8 ] HeadOrDefault 10 TailOrNull [ 1, 2, 6, 3, 4, 9, 8 ] HeadOrDefault 1 TailOrNull [ 2, 6, 3, 4, 9, 8 ] HeadOrDefault 2 TailOrNull [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] Head 2 Tail [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] (values) [ 2, 6, 3, 4, 9, 8 ] Head 1 Tail [ 2, 6, 3, 4, 9, 8 ] HeadOrDefault 2 TailOrNull [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] Head 2 Tail [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] (values) [ 2, 6, 3, 4, 9, 8 ] (values) [ 1, 2, 6, 3, 4, 9, 8 ] Head 10 Tail [ 1, 2, 6, 3, 4, 9, 8 ] HeadOrDefault 1 TailOrNull [ 2, 6, 3, 4, 9, 8 ] HeadOrDefault 2 TailOrNull [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] Head 2 Tail [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] (values) [ 2, 6, 3, 4, 9, 8 ] Head 1 Tail [ 2, 6, 3, 4, 9, 8 ] HeadOrDefault 2 TailOrNull [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] Head 2 Tail [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] (values) [ 2, 6, 3, 4, 9, 8 ] (values) [ 1, 2, 6, 3, 4, 9, 8 ] (values) [ 10, 1, 2, 6, 3, 4, 9, 8 ] Head 7 Tail [ 10, 1, 2, 6, 3, 4, 9, 8 ] HeadOrDefault 10 TailOrNull [ 1, 2, 6, 3, 4, 9, 8 ] HeadOrDefault 1 TailOrNull [ 2, 6, 3, 4, 9, 8 ] HeadOrDefault 2 TailOrNull [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] Head 2 Tail [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] (values) [ 2, 6, 3, 4, 9, 8 ] Head 1 Tail [ 2, 6, 3, 4, 9, 8 ] HeadOrDefault 2 TailOrNull [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] Head 2 Tail [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] (values) [ 2, 6, 3, 4, 9, 8 ] (values) [ 1, 2, 6, 3, 4, 9, 8 ] Head 10 Tail [ 1, 2, 6, 3, 4, 9, 8 ] HeadOrDefault 1 TailOrNull [ 2, 6, 3, 4, 9, 8 ] HeadOrDefault 2 TailOrNull [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] Head 2 Tail [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] (values) [ 2, 6, 3, 4, 9, 8 ] Head 1 Tail [ 2, 6, 3, 4, 9, 8 ] HeadOrDefault 2 TailOrNull [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] Head 2 Tail [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] (values) [ 2, 6, 3, 4, 9, 8 ] (values) [ 1, 2, 6, 3, 4, 9, 8 ] (values) [ 10, 1, 2, 6, 3, 4, 9, 8 ] (values) [ 7, 10, 1, 2, 6, 3, 4, 9, 8 ] Head 5 Tail [ 7, 10, 1, 2, 6, 3, 4, 9, 8 ] HeadOrDefault 7 TailOrNull [ 10, 1, 2, 6, 3, 4, 9, 8 ] HeadOrDefault 10 TailOrNull [ 1, 2, 6, 3, 4, 9, 8 ] HeadOrDefault 1 TailOrNull [ 2, 6, 3, 4, 9, 8 ] HeadOrDefault 2 TailOrNull [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] Head 2 Tail [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] (values) [ 2, 6, 3, 4, 9, 8 ] Head 1 Tail [ 2, 6, 3, 4, 9, 8 ] HeadOrDefault 2 TailOrNull [ 6, 3, 4, 9, 8 ] HeadOrDefault 6 TailOrNull [ 3, 4, 9, 8 ] Head 6 Tail [ 3, 4, 9, 8 ] (values) [ 6, 3, 4, 9, 8 ] Head 2 Tail [ 6, 3, 

In [153]:
let generateSet (maximum:int) (length:int): int list =
    let rec iterate (currentSet: int Set) =
        match currentSet with
        | finished when (finished |> Set.count) >= length -> finished
        | unfinished ->
            let newVal = randomGenerator.Next(0, maximum+1)
            iterate (currentSet |> Set.add newVal)
    iterate ([] |> Set.ofList) |> Set.toList |> randomiseList

In [165]:
let set1 = generateSet 10 5
set1

[ 2, 7, 10, 8, 0 ] HeadOrDefault 2 TailOrNull [ 7, 10, 8, 0 ] HeadOrDefault 7 TailOrNull [ 10, 8, 0 ] HeadOrDefault 10 TailOrNull [ 8, 0 ] HeadOrDefault 8 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method210(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method211(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] Head 0 Tail [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method210(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method211(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] (values) [ 0 ] Head 8 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method210(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method211(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] Head 0 Tail [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method210(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method211(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] (values) [ 0 ] (values) [ 8, 0 ] Head 10 Tail [ 8, 0 ] HeadOrDefault 8 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Mi

In [166]:
solution1 set1

1

In [167]:
let set2 = generateSet 5 5
set2

[ 5, 3, 4, 2, 0 ] HeadOrDefault 5 TailOrNull [ 3, 4, 2, 0 ] HeadOrDefault 3 TailOrNull [ 4, 2, 0 ] HeadOrDefault 4 TailOrNull [ 2, 0 ] HeadOrDefault 2 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method210(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method211(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] Head 0 Tail [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method210(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method211(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] (values) [ 0 ] Head 2 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method210(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method211(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] Head 0 Tail [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method210(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method211(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] (values) [ 0 ] (values) [ 2, 0 ] Head 4 Tail [ 2, 0 ] HeadOrDefault 2 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microso

In [168]:
solution1 set2

1

In [163]:
let set3 = generateSet 5 6
set3

[ 5, 2, 4, 1, 3, 0 ] HeadOrDefault 5 TailOrNull [ 2, 4, 1, 3, 0 ] HeadOrDefault 2 TailOrNull [ 4, 1, 3, 0 ] HeadOrDefault 4 TailOrNull [ 1, 3, 0 ] HeadOrDefault 1 TailOrNull [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] Head 1 Tail [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] (values) [ 1, 3, 0 ] Head 4 Tail [ 1, 3, 0 ] HeadOrDefault 1 TailOrNull [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] Head 1 Tail [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] (values) [ 1, 3, 0 ] (values) [ 4, 1, 3, 0 ] Head 2 Tail [ 4, 1, 3, 0 ] HeadOrDefault 4 TailOrNull [ 1, 3, 0 ] HeadOrDefault 1 TailOrNull [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] Head 1 Tail [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] (values) [ 1, 3, 0 ] Head 4 Tail [ 1, 3, 0 ] HeadOrDefault 1 TailOrNull [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] Head 1 Tail [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] (values) [ 1, 3, 0 ] (values) [ 4, 1, 3, 0 ] (values) [ 2, 4, 1, 3, 0 ] Head 5 Tail [ 2, 4, 1, 3, 0 ] HeadOrDefault 2 TailOrNull [ 4, 1, 3, 0 ] HeadOrDefault 4 TailOrNull [ 1, 3, 0 ] HeadOrDefault 1 TailOrNull [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] Head 1 Tail [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] (values) [ 1, 3, 0 ] Head 4 Tail [ 1, 3, 0 ] HeadOrDefault 1 TailOrNull [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] Head 1 Tail [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] (values) [ 1, 3, 0 ] (values) [ 4, 1, 3, 0 ] Head 2 Tail [ 4, 1, 3, 0 ] HeadOrDefault 4 TailOrNull [ 1, 3, 0 ] HeadOrDefault 1 TailOrNull [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] Head 1 Tail [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] (values) [ 3, 0 ] (values) [ 1, 3, 0 ] Head 4 Tail [ 1, 3, 0 ] HeadOrDefault 1 TailOrNull [ 3, 0 ] HeadOrDefault 3 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] Head 0 Tail [ ] (values) [ 0 ] Head 3 Tail [ 0 ] HeadOrDefault 0 Ta

In [164]:
solution1 set3

6

The following function can be used to time how long a solution takes to run.  It averages the time over multiple runs.

In [170]:
let timeSolution (maximum: int) (length: int) (solution: int list -> int): int*int*int =
    let timeOnce (): int*int*int*int =
        let testSet = generateSet maximum length
        let sw = System.Diagnostics.Stopwatch.StartNew()
        let result = solution testSet
        sw.Stop()
        (maximum, length, result, sw.Elapsed.Microseconds)
    let timingsCount = 100 * length // maximum*length
    let timings = {1..timingsCount} |> Seq.map (fun iter -> // do enough iterations to get a stable result
        let (_, _, _, timing) = timeOnce ()
        timing
    )
    (maximum, length, timings |> Seq.map float |> Seq.average |> round |> int)

In [172]:
timeSolution 200 100 solution1

Item1,200
Item2,100
Item3,3


Let's create timings (in microseconds) for set sizes from 100 to 500.

In [173]:
let lengths = {100..50..500} |> Seq.toList
let maximums = lengths |> List.map (fun n -> 2*n)
let pairs: (int*int) list = List.zip maximums lengths
let timings = pairs |> List.map (fun pair ->
    let (maximum, length) = pair
    let (_, _, timing) = timeSolution maximum length solution1
    timing
)

In [174]:
List.zip lengths timings

[ (100, 3), (150, 5), (200, 7), (250, 9), (300, 12), (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (100, 3) Item1 100 Item2 3 TailOrNull [ (150, 5), (200, 7), (250, 9), (300, 12), (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (150, 5) Item1 150 Item2 5 TailOrNull [ (200, 7), (250, 9), (300, 12), (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (200, 7) Item1 200 Item2 7 TailOrNull [ (250, 9), (300, 12), (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (250, 9) Item1 250 Item2 9 TailOrNull [ (300, 12), (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (300, 12) Item1 300 Item2 12 TailOrNull [ (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (350, 14) TailOrNull [ (400, 16), (450, 19), (500, 21) ] Head (350, 14) Tail [ (400, 16), (450, 19), (500, 21) ] (values) index value 0 (350, 14) 1 (400, 16) 2 (450, 19) 3 (500, 21) Head (300, 12) Item1 300 Item2 12 Tail [ (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (350, 14) TailOrNull [ (400, 16), (450, 19), (500, 21) ] Head (350, 14) Tail [ (400, 16), (450, 19), (500, 21) ] (values) index value 0 (350, 14) 1 (400, 16) 2 (450, 19) 3 (500, 21) (values) index value 0 (300, 12) Item1 300 Item2 12 1 (350, 14) Item1 350 Item2 14 2 (400, 16) Item1 400 Item2 16 3 (450, 19) Item1 450 Item2 19 4 (500, 21) Item1 500 Item2 21 Head (250, 9) Item1 250 Item2 9 Tail [ (300, 12), (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (300, 12) Item1 300 Item2 12 TailOrNull [ (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (350, 14) TailOrNull [ (400, 16), (450, 19), (500, 21) ] Head (350, 14) Tail [ (400, 16), (450, 19), (500, 21) ] (values) index value 0 (350, 14) 1 (400, 16) 2 (450, 19) 3 (500, 21) Head (300, 12) Item1 300 Item2 12 Tail [ (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (350, 14) TailOrNull [ (400, 16), (450, 19), (500, 21) ] Head (350, 14) Tail [ (400, 16), (450, 19), (500, 21) ] (values) index value 0 (350, 14) 1 (400, 16) 2 (450, 19) 3 (500, 21) (values) index value 0 (300, 12) Item1 300 Item2 12 1 (350, 14) Item1 350 Item2 14 2 (400, 16) Item1 400 Item2 16 3 (450, 19) Item1 450 Item2 19 4 (500, 21) Item1 500 Item2 21 (values) index value 0 (250, 9) Item1 250 Item2 9 1 (300, 12) Item1 300 Item2 12 2 (350, 14) Item1 350 Item2 14 3 (400, 16) Item1 400 Item2 16 4 (450, 19) Item1 450 Item2 19 5 (500, 21) Item1 500 Item2 21 Head (200, 7) Item1 200 Item2 7 Tail [ (250, 9), (300, 12), (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (250, 9) Item1 250 Item2 9 TailOrNull [ (300, 12), (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (300, 12) Item1 300 Item2 12 TailOrNull [ (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (350, 14) TailOrNull [ (400, 16), (450, 19), (500, 21) ] Head (350, 14) Tail [ (400, 16), (450, 19), (500, 21) ] (values) index value 0 (350, 14) 1 (400, 16) 2 (450, 19) 3 (500, 21) Head (300, 12) Item1 300 Item2 12 Tail [ (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (350, 14) TailOrNull [ (400, 16), (450, 19), (500, 21) ] Head (350, 14) Tail [ (400, 16), (450, 19), (500, 21) ] (values) index value 0 (350, 14) 1 (400, 16) 2 (450, 19) 3 (500, 21) (values) index value 0 (300, 12) Item1 300 Item2 12 1 (350, 14) Item1 350 Item2 14 2 (400, 16) Item1 400 Item2 16 3 (450, 19) Item1 450 Item2 19 4 (500, 21) Item1 500 Item2 21 Head (250, 9) Item1 250 Item2 9 Tail [ (300, 12), (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (300, 12) Item1 300 Item2 12 TailOrNull [ (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (350, 14) TailOrNull [ (400, 16), (450, 19), (500, 21) ] Head (350, 14) Tail [ (400, 16), (450, 19), (500, 21) ] (values) index value 0 (350, 14) 1 (400, 16) 2 (450, 19) 3 (500, 21) Head (300, 12) Item1 300 Item2 12 Tail [ (350, 14), (400, 16), (450, 19), (500, 21) ] HeadOrDefault (350, 14) TailOrNull [ (400, 16), (450, 19), (500, 21) ] Head (350, 14) Tail [ (400, 16), (450, 19), (500, 21) ] (values) index va

Now let's try fitting an O(n) curve to the results.  I use a semi-calculated, semi-empirical starting point, with a starting order of O(1.5).

In [227]:
let lengthsFloat = lengths |> List.map float |> Array.ofList
let timingsFloat = timings |> List.map float |> Array.ofList

let first<'a> (arr: 'a[]) = arr[0]
let last<'a> (arr: 'a[]) = arr[arr.Length - 1]

// These scaling factors are used to give the parameters p0, p1 and p2 similar magnitudes, which tends to work best with fitting algorithms.
let p2scale = 0.75 // assuming O(1.5) as a starting point
let p1scale = ((last timingsFloat) - (first timingsFloat)) / ((last lengthsFloat) - (first lengthsFloat))
let p0scale = (first timingsFloat) - p1scale * (first lengthsFloat)
(p0scale, p1scale, p2scale)

Item1,-1.5
Item2,0.045
Item3,0.75


In [228]:

let fitFunc (p0:float) (p1:float) (p2:float) (x:float) =
    // printfn "p0 = %f, p1 = %f, p2 = %f, x = %f" p0 p1 p2 x
    (p0*p0scale) + (p1*p1scale) * x**(p2*p2scale)

let p0init = 1
let p1init = 1.
let p2init = 2. // assuming O(1.5) as a starting point
let tolerance = 0.1
let maxIterations = 10000
(p0init, p1init, p2init)

Item1,1
Item2,1
Item3,2


In [229]:
let (p0, p1, p2) = Fit.Curve(lengthsFloat, timingsFloat, fitFunc, p0init, p1init, p2init, tolerance, maxIterations).ToTuple()
(p0, p1, p2)

Item1,1.1981067824780773
Item2,1.0913226719346085
Item3,1.3164394875498766


OK, let's plot the measurements and the fit together.

In [230]:
let measurementCurve = Scatter(x = lengths, y = timings, name="Measurements")

let fitLengths = {(lengths |> List.min)..(lengths |> List.max)} |> Seq.map float |> Array.ofSeq
let fitTimings = fitLengths |> Array.map (fun x -> fitFunc p0 p1 p2 x)
let fitCurve = Scatter(x = fitLengths, y = fitTimings, name="O(n) Fit Curve")

[measurementCurve; fitCurve]
|> Chart.Plot
|> Chart.WithTitle("Solution Time vs. Set Size, Set Density = 50%")
|> Chart.WithXTitle("Set Size")
|> Chart.WithYTitle("Solution Time in Microsec")

<!DOCTYPE html>

In [231]:
let order = p2*p2scale
order

0.9873296156624074

So, approximately, the order of 'solution1' is 1 - not at all what was expected of a "brute force" approach.

## Discussion

Why is the order so close to 1, for what was intended to be a 'brute force' solution?

It may be that list sorting is *so* optimised in F#/.NET that we don't see the cost of it, so you only see the cost of walking up to find the first unused integer, which is naturally O(1).

## Appendix

**Note from Nicolas** If your set length is N, then your solution value must be in the range 0..N.